# **Installing required packages**

In [32]:
!pip install google-generativeai langchain PyPDF2 chromadb faiss-cpu langchain_google_genai langchain-community python-dotenv --quiet

# **Load Gemini api key**

Save your gemini api key in a .env file in drive

In [22]:
import os
from dotenv import load_dotenv

dotenv_path = '/content/drive/My Drive/.env'

load_dotenv(dotenv_path)

google_api_key = os.getenv('GOOGLE_API_KEY')

# **Chat with pdf**



In [4]:
import os
import pickle
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [6]:
def read_pdf(file_path):
    pdf_reader = PdfReader(file_path)

    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [7]:
text = read_pdf('/content/demo.pdf')
print(text)

 
School of Computer Science and Engineering  
 
Sentiment analysis for Telugu  Reviews  
Project report s ubmitted to the partial fulfilment of the course  
CSI4001 – Natural Language Processing and Computational Linguistics  
 
 
 
 
Project report dated :  30-04-2024  
  
   
1. Project Objective  
 
The objective of this project is to develop a sentiment analysis system for 
Telugu language,reviews of electronic products, categorizing them as positive 
(1) or negative (0) by analyzing Telugu user opinions.  
The goal is to understand sentiment towards electronic products in their 
language, aiding in product enhancement for improved customer satisfaction. 
This involves NLP tasks such as tokenization, N -gram analysis for word 
sequences, utilizing BERT models for  contextual understanding, and word -to-
vector conversion to process Telugu text effectively and using some machine 
learning techniques and ,GRU , bidirectional LSTM,LSTM as models to train 
the data on and check which 

In [14]:
def get_text_chunks(text):
  text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=1000,
                  chunk_overlap=200,
                  length_function=len
                  )
  chunks = text_splitter.split_text(text=text)
  return chunks

In [16]:
text_chunks = get_text_chunks(text)
print(text_chunks)

['School of Computer Science and Engineering  \n \nSentiment analysis for Telugu  Reviews  \nProject report s ubmitted to the partial fulfilment of the course  \nCSI4001 – Natural Language Processing and Computational Linguistics  \n \n \n \n \nProject report dated :  30-04-2024  \n  \n   \n1. Project Objective  \n \nThe objective of this project is to develop a sentiment analysis system for \nTelugu language,reviews of electronic products, categorizing them as positive \n(1) or negative (0) by analyzing Telugu user opinions.  \nThe goal is to understand sentiment towards electronic products in their \nlanguage, aiding in product enhancement for improved customer satisfaction. \nThis involves NLP tasks such as tokenization, N -gram analysis for word \nsequences, utilizing BERT models for  contextual understanding, and word -to-\nvector conversion to process Telugu text effectively and using some machine \nlearning techniques and ,GRU , bidirectional LSTM,LSTM as models to train', "vect

In [10]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [23]:
get_vector_store(text_chunks)

In [24]:
def get_conversational_chain():
    prompt_template = """
    Answer the question using the relevant context provided below.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

In [29]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)
    chain = get_conversational_chain()
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    print("Reply:", response["output_text"])

In [30]:
user_question = "what is the objective of the project?"

In [31]:
user_input(user_question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Reply: The objective of this project is to develop a sentiment analysis system for Telugu language,reviews of electronic products, categorizing them as positive (1) or negative (0) by analyzing Telugu user opinions.
